# Проект "Определение уровня английского для фильмов"

В данном проекте необходимо определить, какой уровень английского языка представлен в фильме. Уровень английского языка определяется в соответствии с уровнем Oxford CEFR. Определение уровня осуществляется на основе субтитров к фильму. В дополнение, можно добавлять фильмы и субтитры к ним из открытых источников, поскольку первоначально в дасете 241 фильм.

**План работы:**
1. [Предобработка данных](#data_preprocessing)
2. [Расширение датасета](#dataset_expansion)
3. [Выбор метрики](#metric)
4. [Создание модели](#model)
5. [Анализ результатов](#analysis)

<a name='data_preprocessing'></a>
## 1. Предобработка данных

### 1. 1. Общий файл с фильмами

In [1]:
# Импортирование необходимых библиотек
import numpy as np
import pandas as pd
import PyPDF2
import pysrt
import re

import nltk
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer

from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, classification_report
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier

nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /Users/midle/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/midle/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [2]:
# Импорт файла
df_movies = pd.read_excel(
    '/Users/midle/Desktop/Coding/Data Science/Projects/English_level/English_scores/movies_labels.xlsx',
)

df_movies.drop('id', axis=1, inplace=True)
df_movies.columns = ['movie', 'level']

df_movies.head()

,movie,level
0,10_Cloverfield_lane(2016),B1
1,10_things_I_hate_about_you(1999),B1
2,A_knights_tale(2001),B2
3,A_star_is_born(2018),B2
4,Aladdin(1992),A2/A2+


In [3]:
# Замена промежуточный уровень на нижний
df_movies.loc[df_movies['level'] == 'A2/A2+', 'level'] = 'A2'
df_movies.loc[df_movies['level'] == 'A2/A2+, B1', 'level'] = 'A2'
df_movies.loc[df_movies['level'] == 'B1, B2', 'level'] = 'B1'

df_movies['level_num'] = df_movies['level'].map({
    'A2': 0,
    'B1': 1,
    'B2': 2,
    'C1': 3
})

df_movies.head(5)

,movie,level,level_num
0,10_Cloverfield_lane(2016),B1,1
1,10_things_I_hate_about_you(1999),B1,1
2,A_knights_tale(2001),B2,2
3,A_star_is_born(2018),B2,2
4,Aladdin(1992),A2,0


In [4]:
# Пропущенные значения
df_movies.isna().sum()

movie        0
level        0
level_num    0
dtype: int64

In [5]:
# Распределение количества фильмов по уровням 
df_movies['level'].value_counts().sort_values(ascending=False)

B2    101
B1     63
C1     40
A2     37
Name: level, dtype: int64

### 1. 2. Файлы с субтитрами

In [23]:
# Вычленение то, что не относится к словам
HTML = r'<.*?>'
TAG = r'{.*?}'
COMMENTS = r'[\(\[][A-Za-z ]+[\)\]]'
UPPER = r'[[A-Za-z ]+[\:\]]'
LETTERS = r'[^a-zA-Z\'.,!? ]'
SPACES = r'([ ])\1+'
DOTS = r'[\.]+'
SYMB = r"[^\w\d'\s]"

# Функция для загрузки и лемматизации субтитров
def saving_subs(film_loc):
    # Загрузка субтитров по кодированию
    film_subs = []
    encodings = ['', 'UTF-8-SIG', 'ISO-8859-1', 'utf-8', 'Windows-1252', 'ascii']
    encoding_number = 0

    # Проверка возможности правильности кодирования, иначе - использование другого
    while not film_subs:
        try:
            film_subs = pysrt.open(
                film_loc,
                encoding=encodings[encoding_number]
            )
        except UnicodeDecodeError:
            encoding_number += 1

    # Удаление лишнего и оставление только слов
    fiml_subs = film_subs[1:]
    text = re.sub(TAG, ' ', film_subs.text)
    text = re.sub(HTML, ' ', text)
    text = re.sub(COMMENTS, ' ', text)
    text = re.sub(UPPER, ' ', text)
    text = re.sub(LETTERS, ' ', text)
    text = re.sub(DOTS, r'.', text)
    text = re.sub(SPACES, r'\1', text)
    text = re.sub(SYMB, '', text)
    text = re.sub('www', '', text)
    text = text.lstrip()
    text = text.encode('ascii', 'ignore').decode()
    text = text.lower()

    # Лемматизация слов и добавление их в отдельный список
    film_words = []
    text_list = text.split()
    lemmatizer = WordNetLemmatizer()

    for i in range(len(text_list)):
        word = lemmatizer.lemmatize(text.split()[i])
        # Проверка на наличие слова в списке
        if word not in film_words:
            film_words.append(word)

    # Объединение слов в одну строчку
    film_words = " ".join(film_words)
    return film_words

In [8]:
# Получение субтитров для каждого фильма, воспользовавшись функцией для обработки субтитров
for index, row in df_movies.iterrows():
    print(index)
    folders = ['A2', 'B1', 'B2', 'C1', 'Subtitles']
    folders_num = 0
    dir_found = False
    
    while not dir_found:
        try:
            subs = saving_subs(f"/Users/midle/Desktop/Coding/Data Science/Projects/English_level/English_scores/Subtitles_all/{folders[folders_num]}/{row['movie']}.srt")
        except FileNotFoundError:
            folders_num += 1
        except IndexError:
            dir_found = True
        else:
            df_movies.loc[df_movies['movie'] == row['movie'], 'subs'] = subs
            dir_found = True

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240


In [10]:
# Удаление фильмов, к которым не были найдены субтитры
df_movies = df_movies[(df_movies['subs'] != '') & (df_movies['subs'].notna())]

### 1. 3. Слова по уровню языка

Создадим список со всеми словами, а также отдельный словарь с ключами в виде уровней английского и значениями - индексами начала слов уровня - 1.

In [11]:
# Ссылки на слова (А1-C1)
a1_b2_link = '/Users/midle/Desktop/Coding/Data Science/Projects/English_level/Oxford_CEFR_level/The_Oxford_3000_by_CEFR_level.pdf'
b2_c1_link = '/Users/midle/Desktop/Coding/Data Science/Projects/English_level/Oxford_CEFR_level/The_Oxford_5000_by_CEFR_level.pdf'

In [12]:
# Функция для сохранения слов в список из PDF-файла 
def get_level_words(link):
    reader = PyPDF2.PdfReader(link)
    
    words = ''
    for n in range(len(reader.pages)):
        words += reader.pages[n].extract_text()
        
    words = words.split('\n')
    words = [words[n].split()[0] for n in range(len(words)) if n > 1]
    return words

In [13]:
# Сохранение слов по уровням в отдельные переменные
a1_b2_words = get_level_words(a1_b2_link)
b2_c1_words = get_level_words(b2_c1_link)

# Внесем некоторые корректировки
a1_b2_words[1] = 'a'
a1_b2_words.insert(2, 'an')
b2_c1_words.remove('3000')
b2_c1_words.remove('B2')

In [14]:
# Объединим слова
level_words = a1_b2_words
level_words.extend(b2_c1_words)
level_words[:10]

['A1',
 'a',
 'an',
 'about',
 'above',
 'across',
 'action',
 'activity',
 'actor',
 'actress']

In [15]:
# Создание словаря с ключами в виде уровня английского и значениями в виде индексов начала слов
# данного уровня минус единица
levels = ['A1', 'A2', 'B1', 'B2', 'C1']
levels_dict = {level: level_words.index(level) for level in levels}

print(levels_dict)

{'A1': 0, 'A2': 891, 'B1': 1752, 'B2': 2550, 'C1': 3958}


In [17]:
# Определение количества слов по уровням для каждого фильма
for index, row in df_movies.iterrows():
    # Словарь с подсчетом слов по каждому уровню для отдельного фильма
    film_level_words = {level: [] for level in levels}
    
    # Добавление в словарь слов по их уровням
    for word in row['subs'].split():
        try:
            word_index = level_words.index(word.lower())
            if levels_dict['A1'] < word_index < levels_dict['A2']:
                if word not in film_level_words['A1']:
                    film_level_words['A1'].append(word)
            elif levels_dict['A2'] < word_index < levels_dict['B1']:
                if word not in film_level_words['A2']:
                    film_level_words['A2'].append(word)
            elif levels_dict['B1'] < word_index < levels_dict['B2']:
                if word not in film_level_words['B1']:
                    film_level_words['B1'].append(word)
            elif levels_dict['B2'] < word_index < levels_dict['C1']:
                if word not in film_level_words['B2']:
                    film_level_words['B2'].append(word)
            else:
                if word not in film_level_words['C1']:
                    film_level_words['C1'].append(word)
        except ValueError:
            pass
    
    # Подсчет количества слов по уровням
    levels_words_count = {level: len(film_level_words[level]) for level in levels}
    
    # Добавление количества слов по уровням в отдельные столбцы
    for level in levels:
        df_movies.loc[df_movies['movie'] == row['movie'], level] = levels_words_count[level]

In [26]:
# Получение значения количества слов по уровням для отдельного фильмы
'''
movie = df_movies[df_movies['movie'] == 'Suits.S03E06.720p.HDTV.x264-mSD']
total_words = movie[levels].sum(axis=1)
words_by_previous_levels = 0

for level in levels:    
    if ((movie[level].values + words_by_previous_levels) / total_words >= 0.75).values[0]:
        print(level)
        break
    else:
        words_by_previous_levels += movie[level].values
'''

A2


<a name='dataset_expansion'></a>
## 2. Расширение датасета

<a name='metric'></a>
## 3. Выбор метрики

Имеем дело с задачей мультиклассификации. В связи с этим необходимо использовать соответствующие метрики определения качества моделей. Назначение проекта - помочь учащимся, изучающих английский язык. В связи с этим лучше всего использовать F1-меру, поскольку она сочетает в себе и точность (precision), и полноту (recall). Однако, поскольку мы имеем дело с мультиклассификацией, то необходимо воспользоваться "Макро F1-мерой" (деомнстрирующей среднее значение F1-меры по классам) и "Микро F1-мерой" (глобальная средняя F1-меры, вычисляющая сумму TP, FN и FP).

Таким образом, для определения качества моделей будем использовать **f1_micro** и **f1_macro**.

<a name='model'></a>
## 4. Создание модели

### 4. 1. Разделение выборок

In [19]:
# Разделение датасета на тренировочную и тестовую выборки

# X = df_movies.drop(['movie', 'level', 'level_num'], axis=1)
X = df_movies['subs']
y = df_movies['level_num']

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=df_movies['level_num']
)

In [20]:
# Функция для нахождения предсказаний
def get_y_pred(model):
    vectorizer = TfidfVectorizer(stop_words={'english'})
    vec_col = ['subs']
    scaler = StandardScaler()
    
#     preprocessing = ColumnTransformer([
#         ('vectorizer', vectorizer, vec_col),
#         ('scaler', scaler, levels),
#     ])
    
#     pipe = Pipeline([
#         ('preprocessing', preprocessing),
#         ('model', model)
#     ])
    
    pipe = Pipeline([
        ('vectorizer', vectorizer),
        ('model', model)
    ])
    
    pipe.fit(X_train, y_train)
    y_pred = pipe.predict(X_test)
    return y_pred

In [21]:
def print_f1_score(model, pred):
    # Проверка DecisionTreeClassifier на тестовой выборке
    print(f"{model}:")
    print("\tf1_micro:", f1_score(y_test, pred, average='micro'))
    print("\tf1_macro:", f1_score(y_test, pred, average='macro'))

### 4. 2. DecisionTreeClassifier

In [22]:
model = DecisionTreeClassifier(random_state=42)
y_pred = get_y_pred(model)
print_f1_score(model, y_pred)

DecisionTreeClassifier(random_state=42):
	f1_micro: 0.6595744680851063
	f1_macro: 0.6561016511867905


### 4. 3. RandomForestClassifier

In [ ]:
model = RandomForestClassifier(random_state=42)
y_pred = get_y_pred(model)
print_f1_score(model, y_pred)

### 4. 4. LogisticRegression

In [ ]:
model = LogisticRegression(max_iter=300)
y_pred = get_y_pred(model)
print_f1_score(model, y_pred)

<a name='analysis'></a>
## 5. Анализ результатов